In [20]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [21]:
train_dir = "Cars/Cars"

In [22]:
datagen = ImageDataGenerator(validation_split=0.2, rescale=1./255)

In [23]:
train_gen = datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    subset='training',
    class_mode='categorical'
)

Found 395 images belonging to 28 classes.


In [24]:
val_gen = datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    subset='validation',
    class_mode='categorical'
)

Found 82 images belonging to 28 classes.


In [25]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(224,224,3),
                                               include_top=False,
                                               weights="imagenet")

In [26]:
base_model.trainable = False

In [27]:
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(len(train_gen.class_indices), activation="softmax")
])

In [28]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [29]:
history = model.fit(train_gen, validation_data=val_gen, epochs=5)

Epoch 1/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 33s 1s/step - accuracy: 0.0633 - loss: 3.5369 - val_accuracy: 0.0976 - val_loss: 3.2502
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 13s 1s/step - accuracy: 0.1696 - loss: 2.9219 - val_accuracy: 0.1951 - val_loss: 2.9793
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 13s 1s/step - accuracy: 0.4000 - loss: 2.4587 - val_accuracy: 0.2317 - val_loss: 2.7944
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 13s 1s/step - accuracy: 0.5392 - loss: 2.0898 - val_accuracy: 0.2439 - val_loss: 2.6537
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 14s 1s/step - accuracy: 0.6835 - loss: 1.8011 - val_accuracy: 0.2439 - val_loss: 2.5505


In [30]:
model.save("car_make_model_classifier.h5")

In [31]:
from tensorflow.keras.models import load_model
import numpy as np
from tensorflow.keras.preprocessing import image

# Load the trained model
model = load_model("car_make_model_classifier.h5")

# Get class labels mapping
class_indices = train_gen.class_indices
labels = {v: k for k, v in class_indices.items()}  # reverse dict {0:"BMW", 1:"Audi", ...}


In [32]:
# Get labels from training generator
labels = list(train_gen.class_indices.keys())

def predict_car(img_path):
    # Load image
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img) / 255.0  # normalize [0,1]
    img_array = np.expand_dims(img_array, axis=0)  # add batch dimension
    
    # Predict
    prediction = model.predict(img_array)
    class_id = np.argmax(prediction)
    confidence = np.max(prediction)
    
    print(f"Predicted: {labels[class_id]}  |  Confidence: {confidence:.2f}")
    return labels[class_id], confidence


In [36]:
img_path = "Cars/Cars/BMW,3 Series,GT,/0bf22123-85db-5ab9-b444-9662bc0f941f (1).jpg"
predict_car(img_path)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 193ms/step
Predicted: BMW,3 Series,GT,  |  Confidence: 0.26


('BMW,3 Series,GT,', np.float32(0.2591565))

In [37]:
print(tf.__version__)

2.20.0
